## Data Cleaner

In [69]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold

label_encoder = LabelEncoder()
std_scaler = StandardScaler()
one_hot_encoder = OneHotEncoder()
min_max_scaler = MinMaxScaler()

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

random_seed=12345

In [70]:
# Load data
data = pd.read_csv(os.path.join("dataset", "diabetic_data.csv"), na_values=["?", "Unknown/Invalid"], dtype=str, keep_default_na=False)
data.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,NaN,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,NaN,NaN,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,NaN,NaN,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,NaN,NaN,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,NaN,NaN,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,NaN,NaN,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [71]:
data.describe()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
count,101766,101766,99493,101763,101766,3197,101766,101766,101766,101766,61510,51817,101766,101766,101766,101766,101766,101766,101745,101408,100343,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766,101766
unique,101766,71518,5,2,10,9,8,26,17,14,17,72,118,7,75,39,33,21,716,748,789,16,4,4,4,4,4,4,4,2,4,4,2,4,4,4,4,2,3,1,1,4,4,2,2,2,2,2,2,3
top,2278392,88785891,Caucasian,Female,[70-80),[75-100),1,1,7,3,MC,InternalMedicine,1,0,13,0,0,0,428,276,250,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
freq,1,40,76099,54708,26068,1336,53990,60234,57494,17756,32439,14635,3208,46652,6086,85027,90383,67630,6862,6752,11555,49474,96420,84748,81778,100227,101063,101680,96575,101765,89080,91116,101743,94438,95401,101458,101728,101763,101727,101766,101766,47383,101060,101753,101765,101764,101765,54755,78363,54864


In [72]:
# Calculate NaN percentages and show the invalid features
# In small dataset, any feauture with NaN >30% are considerd invalid
for col in data.columns:
  percent = round(len(data[data[col].isnull()]) / len(data[col]), 3)
  
  if percent > 0.3:
    print(f"{col}:", percent)

weight: 0.969
payer_code: 0.396
medical_specialty: 0.491


In [73]:
print("Total data = ", len(data))
data.drop_duplicates(["patient_nbr"], keep = "first", inplace = True)

Total data =  101766


In [74]:
# Remove nonuseful features
features_to_remove = ["encounter_id", "patient_nbr", "payer_code", "weight", "medical_specialty"]
data = data.drop(columns=features_to_remove)
data.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,25,1,1,41,0,1,0,0,0,250.83,NaN,NaN,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,Caucasian,Female,[10-20),1,1,7,3,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,Caucasian,Male,[30-40),1,1,7,2,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,Caucasian,Male,[40-50),1,1,7,1,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [75]:
# PREPROCESS AGE
# Convert age from categorical to numeric
# Reason: age contain more information in numeric type
replaceDict = {"[0-10)" : 5,
"[10-20)" : 15,
"[20-30)" : 25, 
"[30-40)" : 35, 
"[40-50)" : 45, 
"[50-60)" : 55,
"[60-70)" : 65, 
"[70-80)" : 75,
"[80-90)" : 85,
"[90-100)" : 95}

data["age"] = data["age"].apply(lambda x : replaceDict[x])

In [76]:
# Group some ids mapping to 1
# [6, 8, 9, 13] discharge to home
# [3, 4, 5, 14, 22, 23, 24] discharge to medical facility
# [12, 15, 16, 17] discharge, related to institution
# [19, 20, 21] expired
# [25, 26] unknow or invalid

data["discharge_disposition_id"] = data["discharge_disposition_id"].apply(lambda x : 1 if int(x) in [6, 8, 9, 13] 
                                                                           else ( 2 if int(x) in [3, 4, 5, 14, 22, 23, 24]
                                                                           else ( 10 if int(x) in [12, 15, 16, 17]
                                                                           else ( 11 if int(x) in [19, 20, 21]
                                                                           else ( 18 if int(x) in [25, 26] 
                                                                           else int(x) )))))

data = data[~data.discharge_disposition_id.isin([11,13,14,19,20,21])]

data["admission_type_id"] = data["admission_type_id"].apply(lambda x : 1 if int(x) in [2, 7]
                                                            else ( 5 if int(x) in [6, 8]
                                                            else int(x) ))

data["admission_source_id"] = data["admission_source_id"].apply(lambda x : 1 if int(x) in [2, 3]
                                                            else ( 4 if int(x) in [5, 6, 10, 22, 25]
                                                            else ( 9 if int(x) in [15, 17, 20, 21]
                                                            else ( 11 if int(x) in [13, 14]
                                                            else int(x) ))))

# Process chemical test data
for col in ["metformin", "repaglinide", "nateglinide", "chlorpropamide", "glimepiride", "acetohexamide", "glipizide", "glyburide", "tolbutamide", "pioglitazone", "rosiglitazone", "acarbose", "miglitol", "troglitazone", "tolazamide", "examide", "citoglipton", "insulin", "glyburide-metformin", "glipizide-metformin", "glimepiride-pioglitazone", "metformin-rosiglitazone", "metformin-pioglitazone"]:
    data[col] = data[col].apply(lambda x : 3 if x == "Up" 
                                              else ( 1 if x == "Down"                                                          
                                              else ( 2 if x == "Steady"
                                              else  0)))

# Possible label
data["change"] = data["change"].apply(lambda x : 1 if x == "Ch"
                                                 else 0)

# Possible label
data["diabetesMed"] = data["diabetesMed"].apply(lambda x : 0 if x == "No"
                                                else 1)


data["max_glu_serum"] = data["max_glu_serum"].apply(lambda x : 2 if x == ">200" 
                                                            else ( 3 if x == ">300"                                                          
                                                            else ( 1 if x == "Norm"
                                                            else  0)))

data["A1Cresult"] = data["A1Cresult"].apply(lambda x : 2 if x == ">7" 
                                                         else (3 if  x == ">8"                                                        
                                                         else ( 1 if x == "Norm"
                                                         else  0)))

data["readmitted"] = data["readmitted"].apply(lambda x : 0 if x == "NO" 
                                                         else 1)

data.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,5,5,18,1,1,41,0,1,0,0,0,250.83,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Caucasian,Female,15,1,1,7,3,59,0,18,0,0,0,276,250.01,255,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,1,1,1
2,AfricanAmerican,Female,25,1,1,7,2,11,5,13,2,0,1,648,250,V27,6,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Caucasian,Male,35,1,1,7,2,44,1,16,0,0,0,8,250.43,403,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,1,1,0
4,Caucasian,Male,45,1,1,7,1,51,0,8,0,0,0,197,157,250,5,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,1,0


In [77]:
# PREPROCESS DIAG_1, DIAG_2, DIAG_3
# Fill NaNs with most common values
diags = ["diag_1", "diag_2", "diag_3"]
for diag in diags:
  common_diag = Counter(list(data[diag])).most_common(1)[0][0]
  data[diag] = data[diag].fillna(common_diag)
  print(f"No NaNs in {diag}:", len(data[data[diag].isna()]) == 0)
  
  data[diag] = data[diag].apply(lambda x : "other" if (str(x).find("V") != -1 or str(x).find("E") != -1)  
                                        else ("circulatory" if int(float(x)) in range(390, 460) or int(float(x)) == 785
                                        else ("respiratory" if int(float(x)) in range(460, 520) or int(float(x)) == 786
                                        else ("digestive"   if int(float(x)) in range(520, 580) or int(float(x)) == 787
                                        else ("diabetes"    if int(float(x)) == 250
                                        else ("injury"      if int(float(x)) in range(800, 1000)
                                        else ("musculoskeletal" if int(float(x)) in range(710, 740)
                                        else ("genitourinary"   if int(float(x)) in range(580, 630) or int(float(x)) == 788
                                        else ("neoplasms"       if int(float(x)) in range(140, 240)
                                        else ("pregnecy"        if int(float(x)) in range(630, 680)
                                        else "other"))))))))))

print("\nShow converted diags:")
for diag in diags:
  print(diag)
  print(data[diag].head(), "\n")

No NaNs in diag_1: True
No NaNs in diag_2: True
No NaNs in diag_3: True

Show converted diags:
diag_1
0     diabetes
1        other
2     pregnecy
3        other
4    neoplasms
Name: diag_1, dtype: object 

diag_2
0     diabetes
1     diabetes
2     diabetes
3     diabetes
4    neoplasms
Name: diag_2, dtype: object 

diag_3
0       diabetes
1          other
2          other
3    circulatory
4       diabetes
Name: diag_3, dtype: object 



In [78]:
# Encode diags
print("\nEncode diags into numeric type:")
for diag in diags:
  data[diag] = label_encoder.fit_transform(data[diag])
  print(f"Encoded {diag}")
  print(data[diag].head(), "\n")


Encode diags into numeric type:
Encoded diag_1
0    1
1    7
2    8
3    7
4    6
Name: diag_1, dtype: int32 

Encoded diag_2
0    1
1    1
2    1
3    1
4    6
Name: diag_2, dtype: int32 

Encoded diag_3
0    1
1    7
2    7
3    0
4    1
Name: diag_3, dtype: int32 



In [79]:
# Remove least significant features using spearman CC
numeric_features = ["age", "time_in_hospital", "num_lab_procedures", "num_procedures", 
                    "num_medications", "number_outpatient", "number_emergency", "number_inpatient", 
                    "number_diagnoses",
                    "A1Cresult", "max_glu_serum", "metformin", "repaglinide", "nateglinide",
                    "chlorpropamide", "glimepiride", "acetohexamide", "glipizide", "glyburide", 
                    "tolbutamide", "pioglitazone", "rosiglitazone", "acarbose", "miglitol", "troglitazone", 
                    "tolazamide", "examide", "citoglipton", "insulin", "glyburide-metformin", 
                    "glipizide-metformin", "glimepiride-pioglitazone", "metformin-rosiglitazone", 
                    "metformin-pioglitazone", "change", "diabetesMed"]

rejected_numeric_fts = []

for col in numeric_features :
    rho , pval = scipy.stats.spearmanr(data['readmitted'], data[col])
    if pval < 0.05 : 
        continue
    else : 
        rejected_numeric_fts.append(col)
        
print("Rejected insignificant features")
print(rejected_numeric_fts)
data.drop(rejected_numeric_fts, inplace=True,axis=1)

Rejected insignificant features
['nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'tolbutamide', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone']


c:\Users\tonyh\Desktop\diabetes\venv\lib\site-packages\scipy\stats\_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


In [80]:
# PREPROCESS GENDER
# Convert gender to binary (female: 0, male: 1)
# Remove rows that contain unknown/invalid gender
# Due to very small amount of datapoints

print("Number of Nan gender:", len(data[data["gender"].isna()]), "\n")
print("Encoder gender to binary:")
data["gender"] = label_encoder.fit_transform(data["gender"])
data["gender"]

Number of Nan gender: 3 

Encoder gender to binary:


0         0
1         0
2         0
3         1
4         1
         ..
101754    0
101755    0
101756    0
101758    0
101765    1
Name: gender, Length: 70434, dtype: int32

In [81]:
# PREPROCESS RACE 
# Get races
print("Show races:", data["race"].unique())
# Encode race to numeric type but remain nan
data["race"] = data["race"].apply(lambda x : 0 if x == "Other" 
                                      else ( 1 if x == "Asian"                                                        
                                      else ( 2 if x == "Caucasian"
                                      else ( 3 if x == "AfricanAmerican"
                                      else ( 4 if x == "Hispanic"
                                      else  x)))))
print("Show encoded races:", data["race"].unique())

# Split non NaN and Nana
data_non_nan_race = data.dropna(subset=["race"])
data_nan_race = data[data["race"].isnull()]

# Check amount per subset
print("Non NaN dataset size:", len(data_non_nan_race))
print("NaN dataset size:",len(data_nan_race))

Show races: ['Caucasian' 'AfricanAmerican' nan 'Other' 'Asian' 'Hispanic']
Show encoded races: [ 2.  3. nan  0.  1.  4.]
Non NaN dataset size: 68513
NaN dataset size: 1921


In [82]:
# SCALING CONTINUOUS VALUE FEATURES
features_to_scale = ['age','time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']

data[features_to_scale] = min_max_scaler.fit_transform(data[features_to_scale])

In [83]:
# Train random forest model
y_race = data_non_nan_race["race"].astype("float")
X_race = data_non_nan_race.drop("race", axis=1)
X_train_race, X_test_race, y_train_race, y_test_race = train_test_split(X_race, y_race, test_size=0.2, random_state=random_seed, shuffle=True)

rf = RandomForestClassifier(max_depth=30, random_state=random_seed)
rf.fit(X_train_race, y_train_race)
print("Accuracy:", rf.score(X_test_race, y_test_race))
y_predict_race = rf.predict(data_nan_race.drop("race", axis=1))


Accuracy: 0.7734072830766985


In [84]:
# Recover dataset
data_nan_race.iloc[:, 0] = y_predict_race
data = pd.concat([data_non_nan_race, data_nan_race])
data["race"]

0         2.0
1         2.0
2         3.0
3         2.0
4         2.0
         ... 
101533    2.0
101537    2.0
101659    2.0
101715    2.0
101732    2.0
Name: race, Length: 70434, dtype: float64

In [85]:
# Check non NaNs
print("Check each feature if it still contain nan:")
failed_test = 0
for col in data.columns:
  if len(data[data[col].isnull()]) != 0:
    failed_test += 1
    print(f"{col}", len(data[data[col].isnull()]) == 0)

if failed_test == 0:
  print("All passed!")
else:
  print("Failed!")

Check each feature if it still contain nan:
All passed!


In [86]:
# Check remaining columns
print("Selected features", data.columns)

Selected features Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'glipizide', 'glyburide', 'pioglitazone',
       'rosiglitazone', 'acarbose', 'miglitol', 'insulin', 'change',
       'diabetesMed', 'readmitted'],
      dtype='object')


In [87]:
# Output clean dataset
data.to_csv(path_or_buf=os.path.join("dataset", "clean_diabetic_dataset.csv"), index=False)

# Notes: Splitting data must be performed in main.py